In [2]:
!pip install opencv-python-headless numpy matplotlib


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [5]:
!wget https://github.com/pjreddie/darknet/raw/master/cfg/yolov3.cfg -O yolov3.cfg
!wget https://pjreddie.com/media/files/yolov3.weights -O yolov3.weights
!wget https://github.com/pjreddie/darknet/raw/master/data/coco.names -O coco.names

--2024-11-11 18:16:33--  https://github.com/pjreddie/darknet/raw/master/cfg/yolov3.cfg
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg [following]
--2024-11-11 18:16:33--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2024-11-11 18:16:34 (46.3 MB/s) - ‘yolov3.cfg’ saved [8342/8342]

--2024-11-11 18:16:34--  https://pjreddie.com/media/files/yolov3.weights


In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load YOLOv3 model
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

# Load class names
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

In [7]:

# Function to detect objects using YOLO
def detect_objects(img):
    height, width, channels = img.shape
    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # Get output layer names
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # Forward pass to get detections
    detections = net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []

    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                w = int(obj[2] * width)
                h = int(obj[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    return boxes, confidences, class_ids

# Apply Non-Maximum Suppression
def apply_nms(boxes, confidences, confidence_threshold=0.5, nms_threshold=0.4):
    return cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)

# Draw bounding boxes and labels
def draw_boxes(img, boxes, confidences, class_ids, indices):
    for i in indices.flatten():
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = confidences[i]

        color = (0, 255, 0)
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return img

In [8]:
# Upload an image for testing
from google.colab import files
uploaded = files.upload()

# Load the uploaded image
img = cv2.imread(list(uploaded.keys())[0])  # Use the uploaded file name dynamically
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Detect objects in the image
boxes, confidences, class_ids = detect_objects(img)
indices = apply_nms(boxes, confidences)

# Draw boxes and labels on the image
img_with_boxes = draw_boxes(img, boxes, confidences, class_ids, indices)

# Show the result
plt.imshow(img_with_boxes)
plt.axis('off')
plt.show()